# Proyecto de Tesis

## CNN

In [9]:
import torch
import torchvision

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [11]:
train = torchvision.datasets.MNIST(root = './MNIST/train', train = True, download = True)
test  = torchvision.datasets.MNIST(root = './MNIST/test', train = False, download = True)

9913344it [00:03, 3086951.22it/s]                             


Extracting ./MNIST/train\MNIST\raw\train-images-idx3-ubyte.gz to ./MNIST/train\MNIST\raw



29696it [00:00, 7477580.09it/s]          


Extracting ./MNIST/train\MNIST\raw\train-labels-idx1-ubyte.gz to ./MNIST/train\MNIST\raw



1649664it [00:00, 2262898.49it/s]                             


...

Extracting ./MNIST/test\MNIST\raw\train-labels-idx1-ubyte.gz to ./MNIST/test\MNIST\raw



1649664it [00:00, 2936330.34it/s]                             


Extracting ./MNIST/test\MNIST\raw\t10k-images-idx3-ubyte.gz to ./MNIST/test\MNIST\raw



5120it [00:00, ?it/s]                   

Extracting ./MNIST/test\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./MNIST/test\MNIST\raw

